In [1]:
from cdasws import CdasWs
import numpy as np
import pandas as pd
cdas = CdasWs()

In [25]:
#Check variable names for particular dataset
variables = cdas.get_variables('OMNI_HRO_1MIN')
variables

[{'Name': 'IMF',
  'ShortDescription': '',
  'LongDescription': 'OMNI ID code for the source spacecraft for time-shifted IMF values (see OMNI documentation link for codes)'},
 {'Name': 'PLS',
  'ShortDescription': '',
  'LongDescription': 'OMNI ID code for the source spacecraft  for time-shifted IP plasma values (see OMNI documentation link for codes)'},
 {'Name': 'IMF_PTS',
  'ShortDescription': '',
  'LongDescription': 'Number of fine time scale points in IMF averages'},
 {'Name': 'PLS_PTS',
  'ShortDescription': '',
  'LongDescription': 'Number of fine time scale points in plasma averages'},
 {'Name': 'percent_interp',
  'ShortDescription': '',
  'LongDescription': 'Percent interpolated'},
 {'Name': 'Timeshift',
  'ShortDescription': '',
  'LongDescription': 'Timeshift (seconds)'},
 {'Name': 'RMS_Timeshift',
  'ShortDescription': '',
  'LongDescription': 'RMS Timeshift (seconds)'},
 {'Name': 'RMS_phase',
  'ShortDescription': '',
  'LongDescription': 'RMS, Phase front normal (nT) '}

In [ ]:
#Save DIS-MOMS data to HDF5 file
dates = pd.date_range('2015-07-01 ', '2023-02-01', freq='1M').strftime('%Y-%m-%d %H:%M:%S+0000').tolist()

dis_df = pd.DataFrame([])
for i in np.arange(len(dates)-1):
    dis_df_stage = pd.DataFrame([])
    try:
        data = cdas.get_data('MMS1_FPI_FAST_L2_DIS-MOMS', ['mms1_dis_bulkv_gse_fast', 'mms1_dis_numberdensity_fast', 'mms1_dis_energyspectr_omni_fast'], dates[i], dates[i+1])
    except ConnectionError:
        print('Connection error for ' + dates[i])
        continue
    try:
        dis_df_stage['Epoch_dis'] = data[1]['Epoch']
        dis_df_stage['Vi_xgse'] = data[1]['mms1_dis_bulkv_gse_fast'][:, 0]
        dis_df_stage['Vi_ygse'] = data[1]['mms1_dis_bulkv_gse_fast'][:, 1]
        dis_df_stage['Vi_zgse'] = data[1]['mms1_dis_bulkv_gse_fast'][:, 2]
        dis_df_stage['Ni'] = data[1]['mms1_dis_numberdensity_fast']
        dis_df_stage['SW_table'] = data[1]['mms1_dis_energy_fast'][:,0] >= 190 #Solar wind energy-azimuth table starts at ~190-~210 eV
        dis_df = dis_df.append(dis_df_stage, ignore_index=True)
    except TypeError:
        print('No data for ' + dates[i])
        continue
dis_df['Epoch_dis'] = pd.to_datetime(dis_df['Epoch_dis'], utc=True)
#dis_df.to_hdf('../data/mms_data_test.h5', key = 'dis_raw', mode = 'a')

In [6]:
#Save DES-MOMS data to HDF5 file
dates = pd.date_range('2015-01-01', '2023-04-01', freq='3M').strftime('%Y-%m-%d %H:%M:%S+0000').tolist()

des_df = pd.DataFrame([])
for i in np.arange(len(dates)-1):
    des_df_stage = pd.DataFrame([])
    try:
        data = cdas.get_data('MMS1_FPI_FAST_L2_DES-MOMS', ['mms1_des_numberdensity_fast'], dates[i], dates[i+1])
        des_df_stage['Epoch_des'] = data[1]['Epoch']
        des_df_stage['Ne'] = data[1]['mms1_des_numberdensity_fast']
        des_df = des_df.append(des_df_stage, ignore_index=True)
    except TypeError: #Throws when date range is empty OR too big
        print('Error at: ', dates[i], dates[i+1])
        continue
des_df['Epoch_des'] = pd.to_datetime(des_df['Epoch_des'], utc=True)
#des_df.to_hdf('../data/mms_data_test.h5', key='des_raw', mode = 'a')

Error at:  2015-01-31 00:00:00+0000 2015-04-30 00:00:00+0000
Error at:  2016-04-30 00:00:00+0000 2016-07-31 00:00:00+0000


In [8]:
#Save MEC data to a HDF5 file
dates = pd.date_range('2015-01-01', '2023-04-01', freq='3M').strftime('%Y-%m-%d %H:%M:%S+0000').tolist()

mec_df = pd.DataFrame([])
for i in np.arange(len(dates)-1):
    mec_df_stage = pd.DataFrame([])
    try:
        data = cdas.get_data('MMS1_MEC_SRVY_L2_EPHT89D', ['mms1_mec_r_gsm', 'mms1_mec_r_gse'], dates[i], dates[i+1])
        mec_df_stage['Epoch_mec'] = data[1]['Epoch']
        mec_df_stage['R_xgsm'] = data[1]['mms1_mec_r_gsm'][:, 0]
        mec_df_stage['R_ygsm'] = data[1]['mms1_mec_r_gsm'][:, 1]
        mec_df_stage['R_zgsm'] = data[1]['mms1_mec_r_gsm'][:, 2]
        mec_df_stage['R_xgse'] = data[1]['mms1_mec_r_gsm'][:, 0]
        mec_df_stage['R_ygse'] = data[1]['mms1_mec_r_gsm'][:, 1]
        mec_df_stage['R_zgse'] = data[1]['mms1_mec_r_gsm'][:, 2]
        mec_df_stage
        mec_df = mec_df.append(mec_df_stage, ignore_index=True)
    except TypeError: #Throws when date range is empty OR too big
        print('Error at: ', dates[i], dates[i+1])
        continue
mec_df['Epoch_mec'] = pd.to_datetime(mec_df['Epoch_mec'], utc=True)
mec_df.to_hdf('../data/mms_data_test.h5', key='mec_raw', mode = 'a')

In [15]:
#Save FGM data to a HDF5 file
start = '2022-01-01'
end = '2023-01-01'
load_freq = '1D'
bin_freq = '5s'
end_delt = pd.to_datetime(end) + pd.Timedelta(load_freq) #Get the last date of the date range to ensure the end is included
dates = pd.date_range(start, end_delt, freq=load_freq).strftime('%Y-%m-%d %H:%M:%S+0000').tolist() #1 day intervals from 2015 to 2023 (inclusive)
fgm_df = pd.DataFrame([])
#Load the FGM data for each day and bin it
for i in range(len(dates)-1):
    try:
        fgm_data = pd.DataFrame([])
        data = cdas.get_data('MMS1_FGM_SRVY_L2', ['mms1_fgm_b_gsm_srvy_l2', 'mms1_fgm_b_gse_srvy_l2', 'mms1_fgm_flag_srvy_l2'], dates[i], dates[i+1])
        fgm_data['Epoch_fgm'] = data[1]['Epoch']
        fgm_data['Bx_gsm'] = data[1]['mms1_fgm_b_gsm_srvy_l2'][:, 0]
        fgm_data['By_gsm'] = data[1]['mms1_fgm_b_gsm_srvy_l2'][:, 1]
        fgm_data['Bz_gsm'] = data[1]['mms1_fgm_b_gsm_srvy_l2'][:, 2]
        fgm_data['Bx_gse'] = data[1]['mms1_fgm_b_gse_srvy_l2'][:, 0]
        fgm_data['By_gse'] = data[1]['mms1_fgm_b_gse_srvy_l2'][:, 1]
        fgm_data['Bz_gse'] = data[1]['mms1_fgm_b_gse_srvy_l2'][:, 2]
        fgm_data['B_flag'] = data[1]['mms1_fgm_flag_srvy_l2']
    except TypeError: #Throws when no FGM data to load
        fgm_data = pd.DataFrame([[np.NaN, np.NaN]], columns = ['Epoch_fgm', 'Bx_gsm'])
        print('Date '+str(i+1)+' of '+str(len(dates))+' has no FGM data')
        continue
    except ValueError: #Throws when FGM data is corrupted
        fgm_data = pd.DataFrame([[np.NaN, np.NaN]], columns = ['Epoch_fgm', 'Bx_gsm'])
        print('Date '+str(i+1)+' of '+str(len(dates))+' has corrupted FGM data')
        continue
    times = pd.date_range(dates[i], dates[i+1], freq=bin_freq) #Five second intervals for the day
    bin_subset = pd.IntervalIndex.from_arrays(times[:-1], times[1:], closed='left') #Get the bins for the current day
    fgm_group = fgm_data.groupby(pd.cut(fgm_data['Epoch_fgm'], bin_subset)) #Bin the FGM data
    fgm_binned_stage = fgm_group.mean() #Get the mean of the binned data
    fgm_binned_stage['count'] = fgm_group.count()['Bx_gsm'] #Get the number of data points in each bin
    fgm_binned_stage['Epoch_fgm'] = bin_subset.left #Add the start of the bins as the Epoch
    fgm_df = fgm_df.append(fgm_binned_stage, ignore_index = True) #Add the binned data to the dataframe
    print('Date '+str(i+1)+' of '+str(len(dates)-1)+' ('+dates[i]+') loaded', end='\r')
    del fgm_data #Delete the old data to save memory
    del data #Delete the old data to save memory
#fgm_df.to_hdf('../data/mms_data_test.h5', key='fgm_raw', mode = 'a')

KeyboardInterrupt: 

In [3]:
#Make a combined dataframe with all the data
dis_df = pd.read_hdf('../data/mms_data.h5', key='dis_raw', mode = 'a')
des_df = pd.read_hdf('../data/mms_data.h5', key='des_raw', mode = 'a')
mec_df = pd.read_hdf('../data/mms_data.h5', key='mec_raw', mode = 'a')
fgm_df = pd.read_hdf('../data/mms_data.h5', key='fgm_raw', mode = 'a')
bins = pd.read_hdf('../data/mms_data.h5', key='bins', mode = 'a')

bins_index = pd.IntervalIndex.from_arrays(bins['start'], bins['end'], closed='left') #Make interval index for binning

In [4]:
#Group the mec, des, and dis data by the bins
verbose = True
mec_group = mec_df.groupby(pd.cut(mec_df['Epoch_mec'], bins_index))
if verbose: print('Finished grouping mec')
des_group = des_df.groupby(pd.cut(des_df['Epoch_des'], bins_index))
if verbose: print('Finished grouping des')
dis_group = dis_df.groupby(pd.cut(dis_df['Epoch_dis'], bins_index))
if verbose: print('Finished grouping dis')
fgm_group = fgm_df.groupby(pd.cut(fgm_df['Epoch_fgm'], bins_index))
if verbose: print('Finished grouping fgm')
#Bin the mec, des, and dis data into staging dataframes
mec_binned = mec_group.mean()
des_binned = des_group.mean()
dis_binned = dis_group.mean()
fgm_binned = fgm_group.mean()
if verbose: print('Finished binning data')
#Add the counts in each bin
mec_binned['count_mec'] = mec_group.count()['R_xgsm']
des_binned['count_des'] = des_group.count()['Ne']
dis_binned['count_dis'] = dis_group.count()['Ni']
fgm_binned['count_fgm'] = fgm_group.count()['Bx_gsm']
if verbose: print('Finished adding counts')
#Get the first Epoch in each bin
mec_binned['Epoch_mec'] = mec_group.first()['Epoch_mec']
des_binned['Epoch_des'] = des_group.first()['Epoch_des']
dis_binned['Epoch_dis'] = dis_group.first()['Epoch_dis']
fgm_binned['Epoch_fgm'] = fgm_group.first()['Epoch_fgm']
if verbose: print('Finished adding Epochs')
#Set the index to numbers instead of the bins
mec_binned = mec_binned.reset_index(drop = True)
des_binned = des_binned.reset_index(drop = True)
dis_binned = dis_binned.reset_index(drop = True)
fgm_binned = fgm_binned.reset_index(drop = True)
if verbose: print('Finished resetting index')
#Add the region column
mec_binned['region'] = bins['region'].to_numpy()
des_binned['region'] = bins['region'].to_numpy()
dis_binned['region'] = bins['region'].to_numpy()
fgm_binned['region'] = bins['region'].to_numpy()
if verbose: print('Finished adding region column')

Finished grouping mec
Finished grouping des
Finished grouping dis
Finished grouping fgm
Finished binning data
Finished adding counts
Finished adding Epochs
Finished resetting index
Finished adding region column


In [23]:
#Combine the three dataframes
mms_data = pd.merge(mec_binned.drop(columns = 'region'), des_binned.drop(columns = 'region'), left_index = True, right_index = True)
mms_data = pd.merge(mms_data, dis_binned.drop(columns = 'region'), left_index = True, right_index = True)
mms_data = pd.merge(mms_data, fgm_binned.drop(columns = 'region'), left_index = True, right_index = True)
mms_data['Epoch'] = bins['start'] #Add the start of the bins as the Epoch
mms_data['region'] = bins['region'] #Add the region column
#Drop any rows with NaNs
mms_data = mms_data.dropna()
mms_data = mms_data.reset_index(drop = True) #Reset the index
mms_data[['R_xgsm', 'R_ygsm', 'R_zgsm', 'R_xgse', 'R_ygse', 'R_zgse']] = mms_data[['R_xgsm', 'R_ygsm', 'R_zgsm', 'R_xgse', 'R_ygse', 'R_zgse']]/6378 #Convert the R vectors to be in units of Earth radii
#Check if MMS data was obtained with SW energy-azimuth table
mms_data['Vi_ygse'][mms_data['SW_table']] += -10.7 #Subtract 10.7 km/s to Vy_gse if MMS data was obtained with SW energy-azimuth table
mms_data['Vi_ygse'][mms_data['SW_table']] += -29.0 #Subtract 29.0 km/s to Vy_gse if MMS data was not obtained with SW energy-azimuth table
#Save the combined data
#mms_data.to_hdf('../data/mms_data.h5', key = 'targets', mode = 'a')
print('Finished saving combined data')

c:\Users\conno\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\Users\conno\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\conno\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Finished saving combined data


In [ ]:
import sw_lib as sw
start = pd.to_datetime('2022-01-01')
end = pd.to_datetime('2023-01-01')
dates = pd.date_range(start, end + pd.Timedelta('1D'), freq='1D').strftime('%Y-%m-%d').tolist()

date = start #Initialize the date
ol_df = pd.DataFrame([]) #Initialize the dataframe
for i in np.arange(len(dates)-1):
    staging_df = pd.DataFrame([]) #Initialize the staging dataframe
    try:
        data = cdas.get_data('MMS1_FPI_FAST_L2_DIS-DIST', ['mms1_dis_dist_fast'], dates[i], dates[i+1]) #Get the data
        dist_arr = np.asarray(data[1]['mms1_dis_dist_fast']) #Pull the 3D distribution
        label, probability = sw.olshevsky_class(dist_arr) #Classify the full array
        p_arr = sw.translate_probabilities(probability) #Store the class probabilities
        f_arr = sw.translate_labels(label, classifier = 'olshevsky') #Region flags (see translate_labels)
        staging_df['Epoch'] = data[1]['Epoch'] #Get the time array
        staging_df['region'] = f_arr #Get the region array
        staging_df['MSP_p'] = p_arr[:, 0] #Get the MSP probabilities
        staging_df['MSH_p'] = p_arr[:, 1] #Get the MSH probabilities
        staging_df['SW_p'] = p_arr[:, 2] #Get the SW probabilities
        staging_df['IF_p'] = p_arr[:, 3] #Get the IF probabilities
        ol_df = ol_df.append(staging_df, ignore_index = True) #Append this day to the full dataframe
    except:
        print('Classification failed for date: '+date[i]) #Throw a warning
        continue #Skip this day
ol_df.to_hdf('../data/ol_class_2022', key='ol_class_2022', mode = 'a') #Save our work

In [29]:
import pandas as pd
store = pd.HDFStore('../data/sw_data.h5', mode = 'a')
store['sw_table_list'].to_hdf('sw_table_list.h5', key='sw_table_list', mode = 'a')

In [ ]:
#Correct the old dataset (while the new one goes through growing pains)
mms_data_old = pd.read_hdf('../data/sw_data.h5', key = 'mms_target') #Load the MMS data binned to 100s intervals
des_raw = pd.read_hdf('../data/mms_data.h5', key='des_raw') #Load the DES data
sw_table = pd.read_hdf('../data/sw_table_list.h5', key = 'sw_table_list', mode = 'a') #List of files obtained with SW energy-azimuth table

mms_data_old['Epoch'] = pd.to_datetime(mms_data_old['time'], unit = 's', origin='unix', utc = True) #Convert the timestamp to a datetime object
bins = pd.IntervalIndex.from_arrays(mms_data_old['Epoch'], mms_data_old['Epoch'] + pd.Timedelta('100s'), closed = 'left') #Get the bins for the MMS data
des_group = des_raw.groupby(pd.cut(des_raw['Epoch_des'], bins))
des_stage = des_group.mean()
des_stage.index = mms_data_old.index #Reset the index to be sequential
mms_data_old['Ne'] = des_stage['Ne']

#Check if MMS data was obtained with SW energy-azimuth table
table_bool = mms_data_old['Epoch'].round('2H').isin(sw_table) #Boolean array of MMS data that was obtained with SW energy-azimuth table
mms_data_old['Vy_gse'][(table_bool)&(mms_data_old['regid'] == 2)] += -10.7 #Subtract 10.7 km/s to Vy_gse if MMS data was obtained with SW energy-azimuth table
mms_data_old['Vy_gse'][(~table_bool)&(mms_data_old['regid'] == 2)] += -29.0 #Subtract 29.0 km/s to Vy_gse if MMS data was not obtained with SW energy-azimuth table
mms_data_old.rename(columns = {'Bx_gsm' : 'B_xgsm', 'By_gsm' : 'B_ygsm', 'Bz_gsm' : 'B_zgsm', 
                               'Vx_gse' : 'Vi_xgse', 'Vy_gse' : 'Vi_ygse', 'Vz_gse' : 'Vi_zgse', 
                               'regid' : 'region', 'Px_gse' : 'R_xgse', 'Py_gse' : 'R_ygse', 'Pz_gse' : 'R_zgse'}, inplace = True)
#mms_data_old.to_hdf('../data/sw_data.h5', key = 'mms_target_corrected2', mode = 'a') #Save the corrected MMS data

In [67]:
#Save Wind MFI data to a HDF5 file
dates = pd.date_range('2015-01-01', '2023-04-01', freq='3M').strftime('%Y-%m-%d').tolist()

mfi_df = pd.DataFrame([])
for i in np.arange(len(dates)-1):
    mfi_df_stage = pd.DataFrame([])
    try:
        data = cdas.get_data('WI_H0_MFI', ['BGSM', 'PGSE'], dates[i], dates[i+1])
        mfi_df_stage['Epoch'] = data[1]['Epoch']
        mfi_df_stage['R_xgse'] = data[1]['PGSE'][:, 0]
        mfi_df_stage['R_ygse'] = data[1]['PGSE'][:, 1]
        mfi_df_stage['R_zgse'] = data[1]['PGSE'][:, 2]
        mfi_df_stage['B_xgsm'] = data[1]['BGSM'][:, 0]
        mfi_df_stage['B_ygsm'] = data[1]['BGSM'][:, 1]
        mfi_df_stage['B_zgsm'] = data[1]['BGSM'][:, 2]
        mfi_df = mfi_df.append(mfi_df_stage, ignore_index=True)
    except TypeError: #Throws when date range is empty OR too big
        print('Error at: ', dates[i], dates[i+1])
        continue
mfi_df['Epoch'] = pd.to_datetime(mfi_df['Epoch'], utc=True)
#Set B values to nan if they are equal to the fill value of -1e31
mfi_df['B_xgsm'].where(mfi_df['B_xgsm'] > -1e30, np.nan, inplace=True)
mfi_df['B_ygsm'].where(mfi_df['B_ygsm'] > -1e30, np.nan, inplace=True)
mfi_df['B_zgsm'].where(mfi_df['B_zgsm'] > -1e30, np.nan, inplace=True)
#Set R values to nan if they are equal to the fill value of -1e31
mfi_df['R_xgse'].where(mfi_df['R_xgse'] > -1e30, np.nan, inplace=True)
mfi_df['R_ygse'].where(mfi_df['R_ygse'] > -1e30, np.nan, inplace=True)
mfi_df['R_zgse'].where(mfi_df['R_zgse'] > -1e30, np.nan, inplace=True)
#mfi_df.to_hdf('../data/wind_data_test.h5', key='mfi_raw', mode = 'a')

In [75]:
#Save Wind SWE data to a HDF5 file
dates = pd.date_range('2015-01-01', '2023-04-01', freq='3M').strftime('%Y-%m-%d').tolist()

swe_df = pd.DataFrame([])
for i in np.arange(len(dates)-1):
    swe_df_stage = pd.DataFrame([])
    try:
        data = cdas.get_data('WI_K0_SWE', ['Np', 'V_GSE', 'THERMAL_SPD', 'QF_V', 'QF_Np'], dates[i], dates[i+1])
        swe_df_stage['Epoch'] = data[1]['Epoch']
        swe_df_stage['Ni'] = data[1]['Np']
        swe_df_stage['Vi_xgse'] = data[1]['V_GSE'][:, 0]
        swe_df_stage['Vi_ygse'] = data[1]['V_GSE'][:, 1]
        swe_df_stage['Vi_zgse'] = data[1]['V_GSE'][:, 2]
        swe_df_stage['Vth'] = data[1]['THERMAL_SPD']
        swe_df_stage['vflag'] = data[1]['QF_V']
        swe_df_stage['niflag'] = data[1]['QF_Np']
        swe_df = swe_df.append(swe_df_stage, ignore_index=True)
    except TypeError: #Throws when date range is empty OR too big
        print('Error at: ', dates[i], dates[i+1])
        continue
swe_df['Epoch'] = pd.to_datetime(swe_df['Epoch'], utc=True)
#Remove erroneous Epochs outside downloaded date range (due to CDAS bug)
swe_df['Epoch'].where(swe_df['Epoch'] >= pd.to_datetime(dates[0], utc=True), np.nan, inplace=True)
swe_df['Epoch'].where(swe_df['Epoch'] <= pd.to_datetime(dates[-1], utc=True), np.nan, inplace=True)
#Remove rows with nan Epochs and reset the index
swe_df.dropna(subset=['Epoch'], inplace=True)
swe_df.reset_index(drop=True, inplace=True)
#Set Ni values to nan if they are equal to the fill value of -1e31
swe_df['Ni'].where(swe_df['Ni'] > -1e30, np.nan, inplace=True)
#Set Vi values to nan if they are equal to the fill value of -1e31
swe_df['Vi_xgse'].where(swe_df['Vi_xgse'] > -1e30, np.nan, inplace=True)
swe_df['Vi_ygse'].where(swe_df['Vi_ygse'] > -1e30, np.nan, inplace=True)
swe_df['Vi_zgse'].where(swe_df['Vi_zgse'] > -1e30, np.nan, inplace=True)
#Set Vth values to nan if they are equal to the fill value of -1e31
swe_df['Vth'].where(swe_df['Vth'] > -1e30, np.nan, inplace=True)
#Set vflag values to nan if they are equal to the fill value of -2147483648
swe_df['vflag'].where(swe_df['vflag'] > -2147483648, np.nan, inplace=True)
#Set niflag values to nan if they are equal to the fill value of -2147483648
swe_df['niflag'].where(swe_df['niflag'] > -2147483648, np.nan, inplace=True)
#swe_df.to_hdf('../data/wind_data_test.h5', key='swe_raw', mode = 'a')

In [54]:
def closest_argmin(A, B):
    '''
    Helper function that returns indices of elements in array B closest to each element in array A.
    '''
    L = B.size
    sidx_B = B.argsort()
    sorted_B = B[sidx_B]
    sorted_idx = np.searchsorted(sorted_B, A)
    sorted_idx[sorted_idx==L] = L-1
    mask = (sorted_idx > 0) & \
    ((np.abs(A - sorted_B[sorted_idx-1]) < np.abs(A - sorted_B[sorted_idx])) )
    return sidx_B[sorted_idx-mask]

#Combine the two dataframes
swe_df = pd.read_hdf('../data/wind_data_test.h5', key='swe_raw')
mfi_df = pd.read_hdf('../data/wind_data_test.h5', key='mfi_raw')

#Drop rows with dips in the Epoch
swe_df_new = swe_df.copy()
for i in np.arange(1,len(swe_df)):  #First we hunt dips in the data
    delt = (swe_df['Epoch'][i] - swe_df['Epoch'][i-1]).total_seconds()
    if (delt < 0): #If the time difference is negative, we have a dip!
        swe_df_new.drop(index=i, inplace=True) #Drop the row
    print('Dip Loop ' + str(100*i/len(swe_df))[0:5] + '% complete', end='\r')

max_allowable_delt = 100 #Maximum allowable time difference between consecutive entries
swe_df_new.reset_index(drop=True, inplace=True) #Reset the index
for i in np.arange(1,len(swe_df_new)):  #Then we hunt for large time differences
    delt = (swe_df_new['Epoch'][i] - swe_df_new['Epoch'][i-1]).total_seconds()
    if (delt > max_allowable_delt*2): #If the time difference is too large, we have a gap!
        rows_stage = pd.DataFrame([]) #Initialize the staging dataframe
        rows_stage['Epoch'] = pd.date_range(swe_df_new['Epoch'][i-1], swe_df_new['Epoch'][i], freq='100s')[1:] #Get the new Epochs
        swe_df_new = swe_df_new.append(rows_stage, ignore_index=True) #Append the rows
    print('Time Interpolation Loop ' + str(100*i/len(swe_df_new))[0:5] + '% complete', end='\r')
swe_df_new['interp_flag'] = swe_df_new.isna().any(axis=1) #Get the rows with NaNs and flag them as interpolated
swe_df_new = swe_df_new.sort_values(by=['Epoch']) #Sort the dataframe by Epoch
swe_df_new = swe_df_new.reset_index(drop=True) #Reset the index

#Find elements in mfi_df closest to each Epoch in swe_df
args = closest_argmin(swe_df_new['Epoch'].to_numpy(), mfi_df['Epoch'].to_numpy())
mfi_df_stage = mfi_df.iloc[args] #Get the closest elements
mfi_df_stage.index = swe_df_new.index #Reset the index to be sequential
swe_df_new = pd.merge(swe_df_new, mfi_df_stage, left_index=True, right_index=True) #Merge the dataframes
swe_df_new.rename(columns={'Epoch_x': 'Epoch', 'Epoch_y': 'Epoch_mfi'}, inplace=True) #Rename the Epoch columns
swe_df_new['flag'] = swe_df_new.isna().any(axis=1) #Get the rows with NaNs and flag them as interpolated

#Interpolate the NaNs
swe_df_new['Ni'] = swe_df_new['Ni'].interpolate(method='linear', axis=0)
swe_df_new['Vi_xgse'] = swe_df_new['Vi_xgse'].interpolate(method='linear', axis=0)
swe_df_new['Vi_ygse'] = swe_df_new['Vi_ygse'].interpolate(method='linear', axis=0)
swe_df_new['Vi_zgse'] = swe_df_new['Vi_zgse'].interpolate(method='linear', axis=0)
swe_df_new['Vth'] = swe_df_new['Vth'].interpolate(method='linear', axis=0)
swe_df_new['vflag'] = swe_df_new['vflag'].interpolate(method='linear', axis=0)
swe_df_new['niflag'] = swe_df_new['niflag'].interpolate(method='linear', axis=0)
swe_df_new['R_xgse'] = swe_df_new['R_xgse'].interpolate(method='linear', axis=0)
swe_df_new['R_ygse'] = swe_df_new['R_ygse'].interpolate(method='linear', axis=0)
swe_df_new['R_zgse'] = swe_df_new['R_zgse'].interpolate(method='linear', axis=0)
swe_df_new['B_xgsm'] = swe_df_new['B_xgsm'].interpolate(method='linear', axis=0)
swe_df_new['B_ygsm'] = swe_df_new['B_ygsm'].interpolate(method='linear', axis=0)
swe_df_new['B_zgsm'] = swe_df_new['B_zgsm'].interpolate(method='linear', axis=0)

swe_df_new['Epoch_delta'] = swe_df_new['Epoch'] - swe_df_new['Epoch_mfi'] #Get the difference between the Epochs
mec_data = pd.read_hdf('../data/mms_data.h5', key='mec_raw', mode = 'a')
mec_data.rename(columns={'R_xgse' : 'target_R_xgse', 'R_ygse' : 'target_R_ygse', 'R_zgse' : 'target_R_zgse'}, inplace=True)
mec_data.rename(columns={'R_xgsm' : 'target_R_xgsm', 'R_ygsm' : 'target_R_ygsm', 'R_zgsm' : 'target_R_zgsm'}, inplace=True)
mec_data[['target_R_xgse', 'target_R_ygse', 'target_R_zgse']] = mec_data[['target_R_xgse', 'target_R_ygse', 'target_R_zgse']]/6378 #Convert the R vectors to be in units of Earth radii
mec_data[['target_R_xgsm', 'target_R_ygsm', 'target_R_zgsm']] = mec_data[['target_R_xgsm', 'target_R_ygsm', 'target_R_zgsm']]/6378 #Convert the R vectors to be in units of Earth radii
args = closest_argmin(swe_df_new['Epoch'].to_numpy(), mec_data['Epoch_mec'].to_numpy())
mec_data_stage = mec_data.iloc[args] #Get the closest elements
mec_data_stage.index = swe_df_new.index #Reset the index to be sequential
swe_df_new = pd.merge(swe_df_new, mec_data_stage, left_index=True, right_index=True) #Merge the dataframes
#swe_df_new.to_hdf('../data/wind_data_test.h5', key='wind_combined', mode = 'a') #Save the combined data

In [26]:
import sw_lib as sw
wind_2015 = pd.read_csv('../data/omni_data/wind_ascii/2015wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2016 = pd.read_csv('../data/omni_data/wind_ascii/2016wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2017 = pd.read_csv('../data/omni_data/wind_ascii/2017wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2018 = pd.read_csv('../data/omni_data/wind_ascii/2018wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2019 = pd.read_csv('../data/omni_data/wind_ascii/2019wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2020 = pd.read_csv('../data/omni_data/wind_ascii/2020wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2021 = pd.read_csv('../data/omni_data/wind_ascii/2021wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_2022 = pd.read_csv('../data/omni_data/wind_ascii/2022wind.txt', delim_whitespace=True, names = sw.WIND_ASCII_COLS)
wind_full = wind_2015.append([wind_2016, wind_2017, wind_2018, wind_2019, wind_2020, wind_2021, wind_2022], ignore_index=True)
wind_full = wind_full.drop_duplicates()
wind_full.index = np.arange(len(wind_full)) #reindex
date = []
for i in np.arange(len(wind_full)):
    date.append(pd.to_datetime(str(int(wind_full['year'][i]))+'-'+str(int(wind_full['doy'][i]))+'-'+str(int(wind_full['hour'][i]))+'-'+str(int(wind_full['minute'][i])), format = '%Y-%j-%H-%M', utc=True))
wind_full['Epoch'] = date
wind_full.to_hdf('../data/wind_data.h5', key='wind_omni_bs', mode = 'a')

In [3]:
import sw_lib as sw
mms_data = pd.read_hdf('../data/mms_data.h5', key='targets', mode = 'a')
wind_data = pd.read_hdf('../data/wind_data.h5', key='wind_omni_bs', mode = 'a')

omni_ind = np.zeros(len(mms_data))
shift = np.zeros(len(mms_data))

omni_time = (wind_data['Epoch'] - pd.Timestamp("1970-01-01 00:00:00+0000")) // pd.Timedelta('1s') #Convert to seconds since epoch

nx = wind_data['Phase_n_x'] #Phase front normal vector
ny = wind_data['Phase_n_y']
nz = wind_data['Phase_n_z']

Vx = wind_data['VX_GSE'] #Solar wind velocity
Vy = wind_data['VY_GSE']
Vz = wind_data['VZ_GSE']

denominator = (nx * Vx) + (ny * Vy) + (nz * Vz) #Denominator of time delay function

Rox = wind_data['BSN_X'] * sw.EARTH_RADIUS #Wind position (Convert from Re to km)
Roy = wind_data['BSN_Y'] * sw.EARTH_RADIUS
Roz = wind_data['BSN_Z'] * sw.EARTH_RADIUS

for i in np.arange(len(mms_data)):
    mms_time = (mms_data['Epoch'][i]- pd.Timestamp("1970-01-01 00:00:00+0000")) // pd.Timedelta('1s') #Convert to seconds since epoch
    cut_bool = wind_data['Epoch'].between(mms_data['Epoch'][i] - pd.Timedelta('1D'), mms_data['Epoch'][i]) #Boolean array of omni data within 1 day on either side of MMS data
    min_ind = cut_bool[cut_bool].index.min() #Find the lowest index so we can store the right one later

    Rdx = mms_data['R_xgse'][i] * sw.EARTH_RADIUS #MMS position (Convert from Re to km)
    Rdy = mms_data['R_ygse'][i] * sw.EARTH_RADIUS
    Rdz = mms_data['R_zgse'][i] * sw.EARTH_RADIUS

    delta_t = (nx[cut_bool] * (Rdx - Rox[cut_bool]) +  ny[cut_bool] * (Rdy - Roy[cut_bool]) + nz[cut_bool] * (Rdz - Roz[cut_bool])) / denominator[cut_bool] #OMNI algorithm time delay function
    try:
        omni_ind[i] = np.argmin(np.abs(omni_time[cut_bool] + delta_t - mms_time)) + min_ind #which piece of omni_data corresponds to this adjusted time?
        shift[i] = np.min(np.abs(omni_time[cut_bool] + delta_t - mms_time))
    except ValueError: #Throws when there's no data in omni_data within 1 day on either side of MMS data
        omni_ind[i] = 0 #Unfortunately nan values don't work with pandas indexing. Look for nan timeshifts to drop data later
        shift[i] = np.nan #Unfortunately nan values don't work with pandas indexing. Look for nan timeshifts to drop data later
    print('OMNI Propagation '+str(100*i/len(mms_data))[0:5]+'% Complete', end='\r')
omni_data_shift = wind_data.loc[omni_ind]
omni_data_shift['timeshift'] = shift
omni_data_shift.reset_index(drop=True, inplace=True)
#omni_data_shift.to_hdf('../data/wind_data.h5', key='wind_omni_shift', mode = 'a')

In [2]:
import ks_custom as ksc
#Split up and scale the datasets
in_keys = ['B_xgsm', 'B_ygsm', 'B_zgsm', 'Vi_xgse', 'Vi_ygse', 'Vi_zgse', 'Ni', 'Vth', 'R_xgse', 'R_ygse', 'R_zgse', 'target_R_xgse', 'target_R_ygse', 'target_R_zgse'] #Wind data keys to include in input dataset
tar_keys = ['B_xgsm', 'B_ygsm', 'B_zgsm', 'Vi_xgse', 'Vi_ygse', 'Vi_zgse', 'Ne'] #Targets from MMS dataset to match
window = 70 #Input window length (100s units)
stride = 16 #Stride away from target (100s units)
inter_frac = 0.15 #How many interpolated points in a row to tolerate
flag = 'tdelt' #Whether to cut inputs as a percent of total or length of maximum interpolated stretch of data
ds = ksc.load_dataset('../data/mms_data.h5', '../data/wind_data.h5', 'sw', in_keys, tar_keys, split_frac=0.2, window=window, stride=stride, inter_frac=int(inter_frac*window), flag=flag, conesize = np.pi/2)
mms_data = pd.read_hdf('../data/mms_data.h5', key = 'targets', mode = 'a')
mms_data = ksc.nightside_cut(mms_data, conesize = np.pi/2) #Cut out the nightside data since classification is unreliable there
mms_data = mms_data[mms_data['Vi_xgse'] <= -250] #Cut out erroneous super slow solar wind (maybe it's the magnetosheath?)
mms_sw = mms_data.drop(mms_data[mms_data['region']!=2].index).dropna() #Solar Wind Data
mms_sh = mms_data.drop(mms_data[mms_data['region']!=1].index).dropna() #Magnetosheath Data
mms_cut = mms_sw[ds.ds_mask]
mms_train = mms_cut.iloc[ds.inds_train]
mms_test = mms_cut.iloc[ds.inds_test]

C:\Users\conno\AppData\Roaming\Python\Python37\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)
